In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
PATH = '/content/drive/MyDrive/Brain22/BDT/csvfile/'
csv_false=pd.read_csv(PATH + 'test_false.csv',header=None)
csv_true=pd.read_csv(PATH + 'test_true.csv',header=None)

In [ ]:
import numpy as np

true = csv_true
false = csv_false

x=np.array(true)
y=np.array(false)

X=np.vstack((x, y))

print(X.shape)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 1)
axes.imshow(X[23].reshape(224,224), cmap='gray') 
axes.set_title('Input')
  
#fig.suptitle(f'target = {target_test[21]}')
plt.show()

In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
img_width, img_height = 224, 224

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

if K.image_data_format() == 'channels_first':
    #input_train = input_train.reshape(input_train.shape[0], 1, img_width, img_height)
    input_test = X.reshape(X.shape[0], 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else:
    #input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 1)
    input_test = X.reshape(X.shape[0], img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

input_test = input_test.astype('float32')

input_test = input_test / 255

target_test = to_categorical(label, 2)

**Attentions visualization**

In [ ]:
from keras.models import load_model
#PATH_c = '/gdrive/My Drive/XAI/resnet_fine_tuned_001.h5'
PATH_c = '/content/drive/MyDrive/Brain2/BDTCS/csvfile/BDT35.h5'
new_model = load_model(PATH_c, compile=False)

In [ ]:
%%capture
!pip3 install tf-keras-vis tensorflow

In [ ]:
predict_score = new_model.predict(input_test, verbose=0)

In [ ]:
id = 7

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 1) 
axes.imshow(input_test[id].reshape(224,224), cmap='gray') 
axes.set_title('Input')
  
fig.suptitle(f'target = {label[id]}, Predicted = {label[id]}')
plt.show()

In [ ]:
target_test[id]

In [ ]:
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils.scores import BinaryScore
from tensorflow.keras.applications.resnet50 import preprocess_input


input_image = images = np.asarray([input_test[id]])
#input_image = input_test[0]
print(input_image.shape)
input_class = np.argmax(target_test[id])
#input_class = target_test[0]
print(input_class)

replace2linear = ReplaceToLinear()
score = CategoricalScore([input_class])
#score = input_class
print(score)
# Create Saliency object.
saliency = Saliency(new_model,model_modifier=replace2linear,clone=False)

#X = preprocess_input(input_image)
X = input_image
# Generate saliency map
saliency_map = saliency(score, X)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# saliency_map = normalize(saliency_map)

image_titles = ['Tumor Brain']
# Render
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 4))
# len(ax)
#ax[i].set_title(title, fontsize=16)
ax.imshow(saliency_map[0], cmap='jet')
ax.axis('off')
#plt.tight_layout()
plt.show()

In [ ]:
# Generate saliency map with smoothing that reduce noise by adding noise
saliency_map2 = saliency(score,
                        X,
                        smooth_samples=20, # The number of calculating gradients iterations.
                        smooth_noise=0.20) # noise spread level.

## Since v0.6.0, calling `normalize()` is NOT necessary.
# saliency_map = normalize(saliency_map)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    ax[i].set_title(title, fontsize=14)
    ax[i].imshow(saliency_map2[i], cmap='jet')
    ax[i].axis('off')
plt.tight_layout()
plt.savefig('smoothgrad.png')
plt.show()

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

# Create Gradcam object
gradcam = Gradcam(new_model,
                  model_modifier=replace2linear,
                  clone=False)

# Generate heatmap with GradCAM
cam = gradcam(score,
              X,
              penultimate_layer=-1)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# cam = normalize(cam)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=16)
    ax[i].imshow(images[i].reshape(224,224))
    ax[i].imshow(heatmap, cmap='jet', alpha=0.5) # overlay
    ax[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus

# Create GradCAM++ object
gradcam = GradcamPlusPlus(new_model,
                          model_modifier=replace2linear,
                          clone=False)

# Generate heatmap with GradCAM++
cam = gradcam(score,
              X,
              penultimate_layer=-1)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# cam = normalize(cam)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap2 = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=16)
    ax[i].imshow(images[i].reshape(224,224))
    ax[i].imshow(heatmap2, cmap='jet', alpha=0.5)
    ax[i].axis('off')
plt.tight_layout()
plt.savefig('gradcam_plus_plus.png')
plt.show()

In [ ]:
from tf_keras_vis.scorecam import Scorecam
from tf_keras_vis.utils import num_of_gpus

# Create ScoreCAM object
scorecam = Scorecam(new_model)

# Generate heatmap with ScoreCAM
cam = scorecam(score, X, penultimate_layer=-1)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# cam = normalize(cam)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap3 = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=16)
    ax[i].imshow(images[i].reshape(224,224))
    ax[i].imshow(heatmap3, cmap='jet', alpha=0.5)
    ax[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from tf_keras_vis.scorecam import Scorecam

# Create ScoreCAM object
scorecam = Scorecam(new_model, model_modifier=replace2linear)

# Generate heatmap with Faster-ScoreCAM
cam = scorecam(score,
               X,
               penultimate_layer=-1,
               max_N=10)

## Since v0.6.0, calling `normalize()` is NOT necessary.
# cam = normalize(cam)

# Render
f, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
for i, title in enumerate(image_titles):
    heatmap4 = np.uint8(cm.jet(cam[i])[..., :3] * 255)
    ax[i].set_title(title, fontsize=16)
    ax[i].imshow(images[i].reshape(224,224))
    ax[i].imshow(heatmap4, cmap='jet', alpha=0.5)
    ax[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Render
f, ax = plt.subplots(nrows=1, ncols=7, figsize=(24, 12))

ax[0].set_title("original", fontsize=12)
ax[0].imshow(input_test[id].reshape(224,224), cmap='gray')
ax[0].axis('off')

ax[1].set_title("Vanilla Saliency", fontsize=12)
ax[1].imshow(saliency_map[0], cmap='jet')
ax[1].axis('off')

ax[2].set_title("Smooth Grad", fontsize=12)
ax[2].imshow(saliency_map2[0], cmap='jet')
ax[2].axis('off')

ax[3].set_title("GradCAM", fontsize=12)
ax[3].imshow(input_test[id].reshape(224,224))
ax[3].imshow(heatmap, cmap='jet', alpha=0.5)
ax[3].axis('off')

ax[4].set_title("GradCAM++", fontsize=12)
ax[4].imshow(input_test[id].reshape(224,224))
ax[4].imshow(heatmap2, cmap='jet', alpha=0.5)
ax[4].axis('off')

ax[5].set_title("ScoreCAM", fontsize=12)
ax[5].imshow(input_test[id].reshape(224,224))
ax[5].imshow(heatmap3, cmap='jet', alpha=0.5)
ax[5].axis('off')

ax[6].set_title("Faster ScoreCAM", fontsize=12)
ax[6].imshow(input_test[id].reshape(224,224))
ax[6].imshow(heatmap4, cmap='jet', alpha=0.5)
ax[6].axis('off')

plt.show()